# Practicals for lecture 1.2

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/practicals/Practicals_1.2.ipynb)

## Introduction to `pandas`

#### 1.2.0 Numpy bool; Create and index dataframes

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
# Numpy bool operations
# Take the array of integer numbers below. Use array boolean operations to filter out the numbers that 
# are greater than 5 AND less than 8, OR that are multiple of 7.
np.random.seed(42)
an_array = np.random.randint(0, 10, 100)

an_array[((an_array > 5) & (an_array < 8)) | (an_array % 7 == 0)]

In [ ]:
string_data = ['cabbage', 'artichoke', 'banana', 'avocado', 'apple', 'orange']
int_data = [1, 2, 3, 4, 5, 6]
float_data = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

# Put together the data above in a DataFrame. What happens if you don't specify the index?

df = pd.DataFrame(dict(string_data=string_data, int_data=int_data, float_data=float_data))

In [ ]:
# Select the column of the dataframe containing the strings using the name of the column

df["string_data"]

In [ ]:
# Select the first two rows of the dataframe
df.loc[:1]

In [ ]:
# Select the rows of the dataframe so that the float data is greater than 0.2

df.loc[df["float_data"] > 0.2, "string_data"]

In [ ]:
# Select the rows of the dataframe so that the float data is greater than 0.2 and the int data is less than 5
df.loc[(df["float_data"] > 0.2) & (df["int_data"] < 5)]

In [ ]:
# use the .loc property to select the value of the float data in the row with index 3
df.loc[3, "float_data"]

In [ ]:
# Add a new column to the dataframe containing the following data entries:
new_data = [500, 300, 200, 400, 600, 500]

df["new_col"] = new_data
df


#### 1.2.1 Methods of `pandas` dataframes

In [ ]:
# Here we create a fake dataframe containing the results of an psychological test with 30 subjects.
# Subjects can be left-handed or right-handed.
# The test has 2 measures (reaction time - RT, and accuracy)
def create_data_df():
    np.random.seed(42)
    subject_ability = np.random.uniform(0, 1, 30)
    return pd.DataFrame({'subject': [f"subject_{i}" for i in range(30)],
                       'handedness': np.random.choice(['left', 'right'], 30),
                       'RT': subject_ability*100 + np.random.uniform(0, 50, 30),
                       'accuracy': subject_ability + np.random.normal(0.8, 0.1, 30)})
df = create_data_df()

In [ ]:
# sort the dataframe by RT:
df.sort_values(by="RT")

In [ ]:
# Compute the mean and standard deviation of the RT and accuracy across the dataset:
df[["RT", "accuracy"]].mean()

In [ ]:
# Use indexing to select the RT of the left-handed subjects, and compute its 90% percentile:
lefthand_rt_90perc = df.loc[df["handedness"] == "left", "RT"].quantile(0.9)

In [ ]:
# Now use the percentile to select the accuracy for left-handed subjects with RT above the 90% percentile:
lefthanded_df = df[df["handedness"] == "left"]
lefthanded_df.loc[lefthanded_df["RT"] > lefthand_rt_90perc, "accuracy"]

In [ ]:
# Create a scatter plot of RT vs accuracy for the right-handed subjects:
righthanded_df = df[df["handedness"] == "right"]
righthanded_df.plot(kind="scatter", x="RT", y="accuracy")

In [ ]:
# Take the meteo dataset using the function below
def get_meteo_dataset():
    """Get the meteo dataset from the open-meteo API.
    Note how easy it is to get data from the web with pandas! As long as we give the URL of the csv data, pandas can read it.
    """
    np.random.seed(42)
    URL = "https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41&hourly=temperature_2m,relativehumidity_2m,precipitation,windspeed_10m,winddirection_10m&start_date=2023-02-01&end_date=2023-05-28&format=csv"
    df = pd.read_csv(URL, skiprows=3)  # read the csv file, skipping the first 3 rows (a header)
    df.columns = [col.split(" ")[0] for col in df.columns]  # simplify column names
    df["time"] = pd.to_datetime(df["time"])  # convert the time column to datetime
    df["hour"], df["dayofyear"] = df["time"].dt.hour, df["time"].dt.dayofyear  # extract the hour and day of year
    df["weekdays"] = df["time"].dt.day_name()  # extract the day of the week

    # Here we artificially corrupt some of the data to make it more interesting
    missing_idx = np.random.choice(df.index[:1000], 100)
    df.loc[missing_idx, :] = np.nan
    return df

meteo_df = get_meteo_dataset()

In [ ]:
meteo_df 

In [ ]:
# Plot the temperature and relative humidity for the first 1000 time points. 
# Find points where there's missing data (interrupted line).
meteo_df.loc[:1000, "temperature_2m"].plot()

In [ ]:
# Create a new interpolated temperature column by interpolating the temperature column of the dataframe:
meteo_df["temperature_2m_interp"] = meteo_df["temperature_2m"].interpolate()

In [ ]:
# Make a new plot with the interpolated temperature and the non-interpolated temperature 
# (plot the interpolated first!)
meteo_df.loc[:1000, "temperature_2m_interp"].plot()
meteo_df.loc[:1000, "temperature_2m"].plot()